In [1]:
import pyforest
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('cardio_train.csv')
df.head()

<IPython.core.display.Javascript object>

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [3]:
del df['id']
df['age'] = (df['age']/365).round().astype('int64')
df['weight'] = df['weight'].astype('int64')
df['BMI'] = (df['weight']/(np.power(df['height']/100, 2))).round(2)
del df['weight'], df['height']
df.head(2)
df['gender'] = df['gender'] - 1
df['cholesterol'] = df['cholesterol'] - 1
df['gluc'] = df['gluc'] - 1

<IPython.core.display.Javascript object>

In [4]:
df = df.rename(columns = {'age': 'Age', 'gender': 'Gender', 'ap_hi': 'SBP',
                          'ap_lo': 'DBP','cholesterol': 'Cholesterol_level','gluc': 'Glucose_level', 
                         'smoke': 'Smoker','alco': 'Alcohol_C','active': 'Exercise','cardio': 'Target'})

df = df[['Age', 'Gender', 'BMI','SBP','DBP','Cholesterol_level','Glucose_level','Smoker','Alcohol_C','Exercise', 'Target']]
df.head()

,Age,Gender,BMI,SBP,DBP,Cholesterol_level,Glucose_level,Smoker,Alcohol_C,Exercise,Target
0,50,1,21.97,110,80,0,0,0,0,1,0
1,55,0,34.93,140,90,2,0,0,0,1,1
2,52,0,23.51,130,70,2,0,0,0,0,1
3,48,1,28.71,150,100,0,0,0,0,1,1
4,48,0,23.01,100,60,0,0,0,0,0,0


In [5]:
df.shape

(70000, 11)

In [6]:
outlier_filter = ((df["SBP"]>160) |(df["SBP"]<100)| (df["DBP"]>90)|(df["DBP"]< 60) |(df["BMI"]> 37.39)|(df["BMI"]< 17.75))
df1 = df[~outlier_filter]
df1.shape

(58993, 11)

In [7]:
df1[['Age', 'BMI', 'SBP', 'DBP']].describe()

,Age,BMI,SBP,DBP
count,58993.000000,58993.000000,58993.000000,58993.000000
mean,53.221331,26.662931,124.256980,79.894411
std,6.774501,4.027668,12.787334,7.192900
min,30.000000,17.750000,100.000000,60.000000
25%,48.000000,23.730000,120.000000,80.000000
50%,54.000000,25.950000,120.000000,80.000000
75%,58.000000,29.320000,130.000000,80.000000
max,65.000000,37.390000,160.000000,90.000000


In [9]:
new = df.merge(df1, how = 'outer',indicator=True).loc[lambda x : x['_merge'] == 'left_only']

In [10]:
new.shape

(11007, 12)

In [11]:
new[['Age', 'BMI', 'SBP', 'DBP']].describe()

,Age,BMI,SBP,DBP
count,11007.000000,11007.000000,11007.000000,11007.000000
mean,53.967657,32.342485,153.258654,186.328518
std,6.680995,11.036754,386.358115,464.862176
min,39.000000,3.470000,-150.000000,-70.000000
25%,50.000000,25.640000,123.000000,80.000000
50%,54.000000,30.850000,140.000000,100.000000
75%,60.000000,38.370000,160.000000,100.000000
max,65.000000,298.670000,16020.000000,11000.000000
